### Melon 100 Chart 
* 100곡의 노래의 제목, Song ID추출 (정규표현식 사용)
* Song ID로 상세페이지를 100번 요청해서 노래 상세정보 추출
* 100곡의 상세정보를 json file에 저장한다.
* json file을 Pandas의 DataFrame 객체에 저장해서 표 데이터를 만들기
* 표 데이터를 MariaDB에 저장하기

#### 1.100곡의 노래의 제목, Song ID추출 (정규표현식 사용)

In [25]:
import requests
from bs4 import BeautifulSoup
import re

url = 'https://www.melon.com/chart/index.htm'
req_header = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'
}

# 요청을 보내기
res = requests.get(url, headers=req_header)
print(res.status_code)
# 응답결과가 정상이면
if res.ok:
    # 응답 객체에 포함된 text를 추출
    html = res.text
    # 추출한 text를 파싱하기 위한 BeautifulSoup 객체생성
    soup = BeautifulSoup(html, 'html.parser')
    # <div id='tb_list'> div 태그 선택 ,1개
    print(len(soup.select('div#tb_list')))
    # <div id='tb_list'> 하위에 있는 tr 태그 선택 ,101개
    print(len(soup.select('div#tb_list tr')))
    # <div id='tb_list'> 하위에 있는 tr 하위에 <div class='wrap_song_info'> , 200개
    print(len(soup.select('div#tb_list tr .wrap_song_info')))
    print(len(soup.select('div.wrap_song_info')))
    # <div id='tb_list'> 하위에 있는 tr 하위에 <div class='wrap_song_info'> 하위에 a
    print(len(soup.select('div#tb_list tr .wrap_song_info a'))) #458
    print(len(soup.select('div.wrap_song_info a'))) #458
    # <div id='tb_list'> 하위에 있는 tr 하위에 <div class='wrap_song_info'> 
    # 하위에 <a href='playSong'> 
    print(len(soup.select('div#tb_list tr .wrap_song_info a[href*=\'playSong\']')))
    
    atag_list = soup.select("div#tb_list tr .wrap_song_info a[href*='playSong']")
    print('-------------')
    print(type(atag_list), type(atag_list[2]), atag_list[2].text)
    
    # 100곡 노래 저장할 list
    song_list = []
    for idx, atag in enumerate(atag_list, 1):
        # 1곡 노래 저장할 dict
        song_dict = {}
        
        # a tag 의 content
        title = atag.text
        # a tag의 href 속성 값
        href = atag['href']
        # 정규표현식 패턴으로 숫자를 검색
        matched = re.search(r'(\d+)\);', href)
        #print(matched, matched.group(0), matched.group(1))
        song_id = matched.group(1)
        #print(idx, title, song_id)
        song_url = f'https://www.melon.com/song/detail.htm?songId={song_id}'
        
        #추출한 song title과 url를 song_dict에 저장
        song_dict['title'] = title
        song_dict['url'] = song_url
        #1개의 노래정보를 담고 있는 song_dict를 song_list에 추가
        song_list.append(song_dict)

print(len(song_list))        
song_list[0:3]

200
1
101
200
200
458
458
100
-------------
<class 'bs4.element.ResultSet'> <class 'bs4.element.Tag'> Dynamite
100


[{'title': 'VVS (Feat. JUSTHIS) (Prod. GroovyRoom)',
  'url': 'https://www.melon.com/song/detail.htm?songId=33077590'},
 {'title': '밤하늘의 별을(2020)',
  'url': 'https://www.melon.com/song/detail.htm?songId=33061995'},
 {'title': 'Dynamite',
  'url': 'https://www.melon.com/song/detail.htm?songId=32872978'}]

### 2. Song ID로 상세페이지를 100번 요청해서 노래 상세정보 추출

In [26]:
import requests
from bs4 import BeautifulSoup
import re

req_header = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'
}

#노래 100곡의 상세정보를 저장할 list
song_detail_list = []

for song in song_list:
    #노래 1곡의 상세정보를 저장할 dict
    song_detail_dict = {}
    
    #print(song['title'], song['link'])
    song_url = song['url']
    res = requests.get(song_url, headers=req_header)
    print(res.status_code)
    if res.ok:
        html = res.text
        soup = BeautifulSoup(html,'html.parser')
        #노래제목
        song_detail_dict['곡명'] = song['title']
        #가수
        song_detail_dict['가수'] = \
            soup.select('a[href*="goArtistDetail"] span')[0].text
        #앨범
        song_detail_dict['앨범'] = soup.select('div.meta dd')[0].text
        #발매일
        song_detail_dict['발매일'] = soup.select('div.meta dd')[1].text
        #장르
        song_detail_dict['장르'] = soup.select('div.meta dd')[2].text
        #가사
        lyric = soup.select('div#d_video_summary')[0].text
        #정규표현식 Pattern 생성
        regexp = re.compile(r'[\n\r\t]')     
        #Pattern과 매칭되는 문자열로 empty string으로 치환해라
        song_detail_dict['가사'] = regexp.sub('', lyric.strip())
        
        #각각의 Song정보를 담고 있는 dict를 list에 저장
        song_detail_list.append(song_detail_dict)
#         print('-----')
#         print(song_detail_dict)    

print(len(song_detail_list))
print(song_detail_list[0:3])

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
100
[{'곡명': 'VVS (Feat. JUSTHIS) (Prod. GroovyRoom)', '가수': '미란이 (Mirani)', '앨범': '쇼미더머니 9 Episode 1', '발매일': '2020.11.21', '장르': '랩/힙합', '가사': "보여줘야겠어내가 망할 거랬던 놈들에게도내가 잘될 거라 했던 너에게도다 할게 최선갈아끼워 다 새 거My life's shining like a VVS VVS내 삶은 빛나 VVS VVS보란 듯 나와 TV에Now you see me babe내 삶은 빛나 VVS VVSSkrt-skrtHey new water new vv난 알바 째고 무대 위Yeah go get it go get it가사 위 가난이 빛나지안 가 무한리필살아봐야겠어 내 빌딩 Yeah개 같던 세상의뒤통수 치러 왔지더 크게 Callin’ ma name모두 날 보고 놀래미란이가 TV에떼버려 Tag사 새롭게 Yeah yeah타고 비행Skrr skrr 난 올라가Skrr skrr 난 빛이 나내가 뭐라 했어 Mom꺼내겠다고 포차맨 밑바닥의 소녀엄마의 술병이 날 만들어허기져 이를 꽉 물어Chit chat bout me 덤벼 겁쟁이 너VVS on ma neck 꿈

#### 3. 100곡의 상세정보를 json file에 저장하고 읽어온다.

In [27]:
import json

with open('data/songs.json', 'w', encoding='utf-8') as file:
    json.dump(song_detail_list, file)

In [31]:
with open('data/songs.json','r',encoding='utf-8') as file:
    contents = file.read()
    songs_json = json.loads(contents)

print(len(songs_json), type(songs_json)) 
# [{},{},{}]
#print(songs_json[0])

100 <class 'list'>
{'곡명': 'VVS (Feat. JUSTHIS) (Prod. GroovyRoom)', '가수': '미란이 (Mirani)', '앨범': '쇼미더머니 9 Episode 1', '발매일': '2020.11.21', '장르': '랩/힙합', '가사': "보여줘야겠어내가 망할 거랬던 놈들에게도내가 잘될 거라 했던 너에게도다 할게 최선갈아끼워 다 새 거My life's shining like a VVS VVS내 삶은 빛나 VVS VVS보란 듯 나와 TV에Now you see me babe내 삶은 빛나 VVS VVSSkrt-skrtHey new water new vv난 알바 째고 무대 위Yeah go get it go get it가사 위 가난이 빛나지안 가 무한리필살아봐야겠어 내 빌딩 Yeah개 같던 세상의뒤통수 치러 왔지더 크게 Callin’ ma name모두 날 보고 놀래미란이가 TV에떼버려 Tag사 새롭게 Yeah yeah타고 비행Skrr skrr 난 올라가Skrr skrr 난 빛이 나내가 뭐라 했어 Mom꺼내겠다고 포차맨 밑바닥의 소녀엄마의 술병이 날 만들어허기져 이를 꽉 물어Chit chat bout me 덤벼 겁쟁이 너VVS on ma neck 꿈 앞에녹슨 팔찌 버려 문 앞에구제 벨트 아직 허리에원망하던 과거와 춤출래인디고 카니발 Skrt-skrt목숨은 다 꺼내놓고자판기 밑 먼지 덕지덕지 붙은동전까지 털어-털어여기까지 왔지What you know about me내 속 이야기 털어놓으려지갑 털어 차린 술상 위넌 폰만 바라봤잖아이제 그 스크린에서 날 봐라 I'ma먹고서 Fly 너가 나갈 건 삔또지악플들이 돈과 함께 쏟아지니Got a nice ring to it꼬우면 네 오빠인지 아저씨인지한테일러바치고 한 판 더 뜨자 해 빛이나는 Solo라니까 뭘 그렇게 재니다 드루와 내 패기170에 60kg도 안 되지만 국보 1호 MCThe Q's certified new kingin the rap내가 TV에 쬐끔 나왔다고연예인 된 줄 알았다는

#### 4. json file을 Pandas의 DataFrame 객체에 저장해서 표 데이터를 만들기

In [34]:
import pandas as pd

#DataFrame 객체생성
song_df = pd.DataFrame(columns=['곡명','가수','앨범','발매일','장르','가사'])
#song_df

for song in songs_json:
    # dict객체를 Series 객체에 저장
    series_obj = pd.Series(song)
    # Series 객체를 DataFrame 객체에 append 합니다.
    song_df = song_df.append(series_obj, ignore_index=True)

song_df.head()    

,곡명,가수,앨범,발매일,장르,가사
0,VVS (Feat. JUSTHIS) (Prod. GroovyRoom),미란이 (Mirani),쇼미더머니 9 Episode 1,2020.11.21,랩/힙합,보여줘야겠어내가 망할 거랬던 놈들에게도내가 잘될 거라 했던 너에게도다 할게 최선갈아...
1,밤하늘의 별을(2020),경서,밤하늘의 별을(2020),2020.11.14,발라드,밤하늘의 별을 따서 너에게 줄래 너는 내가 사랑하니까 더 소중하니까 오직 너 아니면...
2,Dynamite,방탄소년단,Dynamite (DayTime Version),2020.08.24,댄스,Cos ah ahI’m in the stars tonightSo watch me b...
3,잠이 오질 않네요,장범준,잠이 오질 않네요,2020.10.24,발라드,당신은 날 설레게 만들어조용한 내 마음 자꾸만 춤추게 해얼마나 얼마나 날 떨리게 하...
4,"내일이 오면 (Feat. 기리보이, BIG Naughty (서동현))",릴보이 (lIlBOI),쇼미더머니 9 Episode 3,2020.12.05,랩/힙합,See I just wannashare the vibe with ya우리가 10년 ...


In [36]:
song_df.tail()

,곡명,가수,앨범,발매일,장르,가사
95,Blue & Grey,방탄소년단,BE,2020.11.20,발라드,Where is my angel하루의 끝을 드리운Someone come and sa...
96,12월의 어느 겨울…,윤도 (YoonDo),12월의 어느 겨울…,2020.12.26,발라드,십이월의 어느 겨울밤하얀 첫눈이 올 때쯤내려오는 눈꽃을 보며걷고 싶어 너와 함께온통...
97,내 방을 여행하는 법,방탄소년단,BE,2020.11.20,R&B/Soul,떠나볼까 let me fly to my시선을 낮추고 어디든 막 zoom지금 나와 l...
98,요즘,양다일,요즘,2020.12.29,발라드,해가 질 때쯤에눈을 뜨고 나면다가오는 어둠 속에나만 홀로 깨어 있는 듯해고요한 거리...
99,크리스마스니까,성시경,Jelly Christmas 2012 HEART PROJECT,2012.12.06,발라드,오늘은 뭔가 좀 다른 기분 Feeling 이젠 때가 된 것 같은 느낌 Love U ...


In [40]:
song_df.sample(3)

,곡명,가수,앨범,발매일,장르,가사
24,뿌리 (Feat. JUSTHIS) (Prod. GroovyRoom),Khundi Panda,쇼미더머니 9 Episode 3,2020.12.05,랩/힙합,뿌리부터 불이 붙어우린 뿌리 그 뿌리 깊숙이뿌리까지 뿌리뿌리부터 불이 붙어우린 뿌리...
16,에잇(Prod.&Feat. SUGA of BTS),아이유,에잇,2020.05.06,록/메탈,So are you happy nowFinally happy now are you뭐...
7,취기를 빌려 (취향저격 그녀 X 산들),산들,취기를 빌려 (취향저격 그녀 X 산들),2020.07.20,발라드,언제부턴가 불쑥내 습관이 돼버린 너혹시나 이런 맘이어쩌면 부담일까널 주저했어언제부턴...


In [46]:
print('shape :', song_df.shape)
print('colums :', song_df.columns)
print('index :', song_df.index)
print(type(song_df.values))
#print('values : ', song_df.values[0:1]) #2차원배열

shape : (100, 6)
colums : Index(['곡명', '가수', '앨범', '발매일', '장르', '가사'], dtype='object')
index : RangeIndex(start=0, stop=100, step=1)
<class 'numpy.ndarray'>


In [48]:
song_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   곡명      100 non-null    object
 1   가수      100 non-null    object
 2   앨범      100 non-null    object
 3   발매일     100 non-null    object
 4   장르      100 non-null    object
 5   가사      100 non-null    object
dtypes: object(6)
memory usage: 4.8+ KB


In [51]:
# 1개의 열(컬럼) 선택
print(type(song_df['가수']))
song_df['가수']

<class 'pandas.core.series.Series'>


0     미란이 (Mirani)
1               경서
2            방탄소년단
3              장범준
4     릴보이 (lIlBOI)
          ...     
95           방탄소년단
96     윤도 (YoonDo)
97           방탄소년단
98             양다일
99             성시경
Name: 가수, Length: 100, dtype: object

In [61]:
# 가수별로 Row Counting
song_df['가수'].value_counts().head(10)

방탄소년단                7
아이유                  5
릴보이 (lIlBOI)         4
폴킴                   4
머쉬베놈 (MUSHVENOM)     3
장범준                  2
이승기                  2
오마이걸 (OH MY GIRL)    2
BLACKPINK            2
미란이 (Mirani)         2
Name: 가수, dtype: int64

In [56]:
#장르별로 Row Counting
song_df['장르'].value_counts()

발라드               27
랩/힙합              19
댄스                16
POP               11
R&B/Soul           6
발라드, 국내드라마         5
발라드, 인디음악          4
록/메탈               3
R&B/Soul, 인디음악     2
록/메탈, 국내드라마        2
랩/힙합, 인디음악         2
성인가요               1
인디음악, 록/메탈         1
인디음악, 포크/블루스       1
Name: 장르, dtype: int64

In [58]:
# unique 한 장르명 가져오기
song_df['장르'].unique()

array(['랩/힙합', '발라드', '댄스', '인디음악, 포크/블루스', 'POP', '발라드, 인디음악', '록/메탈',
       '록/메탈, 국내드라마', '랩/힙합, 인디음악', '발라드, 국내드라마', 'R&B/Soul, 인디음악',
       'R&B/Soul', '인디음악, 록/메탈', '성인가요'], dtype=object)

In [59]:
song_df['가수'].unique()

array(['미란이 (Mirani)', '경서', '방탄소년단', '장범준', '릴보이 (lIlBOI)', '임창정',
       'BLACKPINK', '산들', '스탠딩 에그', 'Jawsh 685', '규현 (KYUHYUN)', '적재',
       'AKMU (악동뮤지션)', '아이유', '박진영', '벤', '노을', '오반 (OVAN)',
       'Khundi Panda', '마마무 (Mamamoo)', '폴킴', '조정석', '창모 (CHANGMO)',
       '오마이걸 (OH MY GIRL)', '순순희', '환불원정대', '스윙스', '경서예지', 'TWICE (트와이스)',
       '마크툽 (MAKTUB)', '백지영', '화사 (Hwa Sa)', '이승기', 'Maroon 5',
       '가호 (Gaho)', '전상근', '이예준', '머쉬베놈 (MUSHVENOM)', '이하이', '신예영',
       '제시 (Jessi)', 'Sia', '잔나비', 'Mariah Carey', '태연 (TAEYEON)',
       'Tones And I', '황인욱', '블루 (BLOO)', 'Ariana Grande', 'Anne-Marie',
       '먼데이 키즈 (Monday Kiz)', '비', 'IZ*ONE (아이즈원)', '엠씨더맥스 (M.C the MAX)',
       '송하예', '전미도', '임영웅', '#안녕', '정준일', 'Dua Lipa', '허각', '지코 (ZICO)',
       '싹쓰리 (유두래곤, 린다G, 비룡)', '청하', '원슈타인', 'Etham', 'Crush',
       'Conan Gray', 'Lauv', 'aespa', '윤도 (YoonDo)', '양다일', '성시경'],
      dtype=object)